In [60]:
from __future__ import division
import math, random
import matplotlib.pyplot as plt

def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""
    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z

# Chapter 7. Hypothesis and Inference

What will we do with all this statistics and probability theory?  
The science part of data science frequently involves forming and testing hypotheses about our data and the processes that generate it.  

## Statistical Hypothesis Testing

For our purposes, [hypotheses](https://en.wikipedia.org/wiki/Hypothesis) are assertions like "this coin is fair" that can be translated into statistics about data.  
In the classical setup, we have a null hypothesis $H_0$ that represents some default position, and an alternative hypothesis $H_1$ that we would like to compare it with.  
We then use statistics to decide whether we can reject $H_0$ as false or not.  

## Example: Flipping a Coin

Imagine that we have a coin and we want to test whether it's fair or not.  
We make the assumption that the coin has some probability $p$ of landing heads up, so our null hypothesis is that the coin is fair, or $p=0.5$.  
The null hypothesis is tested against the alternative hypothesis $p\neq0.5$.

Our test will involve flipping the coin some number $n$ times and counting the number of heads $X$.  
Each coin flip is a [Bernoulli trial](https://en.wikipedia.org/wiki/Bernoulli_trial), which means that X is a Binomial(n,p) random variable that can be approximated using the normal distribution that was covered in Chapter 6.

In [61]:
def normal_approximation_to_binomial(n, p):
    """ finds mu and sigma corresponding to a Binomial(n, p) """
    mu = p * n
    sigma = math.sqrt(p *(1 - p) * n)
    return mu, sigma

Whenever a random variable follows a normal distribution, we can use <code>normal_cdf</code> to figure out the probability that its realized value lies within (or outside) a particular interval.

In [62]:
# The normal_cdf is the probability that the variable is below a threshold
normal_probability_below = normal_cdf

# therefore, if it's not below the threshold, it's above the threshold
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# less than hi and not less than lo gives us between
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# and if it's not between, it's outside
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

We can also do the reverse -- find either the nontail region or the (symmetric) interval around the mean that accounts for a certain level of likelihood.  
For example, if we want to find an interval centerd at the mean and containing 60% probability, then we find the cutoffs where the upper and lower tails each contain 20% of the probability:

In [63]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """ returns the z for which P(Z <= z) = probability """
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """ returns the z for which P(Z >= z) = probability """
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """ returns the symmetric (about the mean) bounds that contain the specified probability """
    tail_probability = (1 - probability) / 2
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In particular, let's say that we choose to flip the coin $n=1000$ times.  
If our hypothesis of fairness is true, $X$ should be distributed approximately normally with mean 50 and standard deviation 15.8:

In [64]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

We need to make a decision about significance --  
How willing are we to make a $type \;1\ error$ (false positive), in which we reject $H_0$ (the null hypothesis) even though it's true?
For reasons lost to the annals of history (meaning it's common practice but no one can agree why), this willingness is often set at 5% or 1%.  
We're going with 5%.

Consider the test that rejects $H_0$ if $X$ falls outside the bounds given by:

In [65]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

Assuming $p$ really equals 0.5 (meaning $H_0$ is true), there is just a 5% chance that we observe an x that lis outside this interval, which is the exact significance that we wanted.  
Said differently, if $H_0$ is true, then, approximately 19 times out of 20, this test will give the correct result.  


We are also often interested in the $power$ of a test, which is the probability of not making a $type \;2\ error$ (false negative), in which we fail to reject $H_0$ even though it is false.  
In order to measure this, we have to specify what exactly $H_0$ being false $means$.  
In other words, knowing merely that $p$ is not 0.5 doesn't tell you very much about the distribution of X.  
In particular, let's check what happens if $p$ is really 0.55, so that the coin is slightly biased towards heads.

In that case, we can calculate the power of the test with:

In [66]:
# 95% bounds based on the assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject the null hypothesis, 
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(type_2_probability)
print(power)
power

0.113451998705
0.886548001295


0.8865480012953671

Imagine instead that our null hypothesis was that the coin is not biased towards heads, or that $p\le0.5$.  
In that case we want a <em>one-sided test</em> that rejects the null hypothesis when X is much larger than 50 but not when X is smaller than 50.  
So, let's do a 5%-significance test using <code>normal_probability_below</code> to find the cutoff below which 95% of the probability lies:

In [67]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi  # is 526 (< 531, since we need more probability in the upper tail)

526.0073585242053

In [68]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.9363794803307173

This is a more powerful test, since it no longer rejects $H_0$ when X is below 469 (which is unlikely to happen if $H_1$ is true)  
and instead rejects $H_0$ when X is between 526 and 531 (which is somewhat likely to happen if $H_1$ is true).

## p-values

An alternative way of thinking about the preceding test involves p-values.  
Instead of choosing bounds based on some probability cutoff, we compute the probability -- assuming $H_0$ is true -- that we would see a value at least as extreme as the one we actually observed.

For our two-sided test of whether the coin is fair, we compute: 

In [69]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2 * normal_probability_below(x, mu, sigma)

# If we were to see 530 heads, we would compute:
two_sided_p_value(529.5, mu_0, sigma_0)  # mu_0 is 500.0

0.06207721579598857

Why did we choose 529.5 instead of 530?  
This is called a [continuity correction](https://en.wikipedia.org/wiki/Continuity_correction).  
It reflects the fact that <code>normal_probability_between(529.5, 530.5, mu_0, sigma_0)</code> is a better estimate of the probability of seeing 530 heads than <code>normal_probability_between(530, 531, mu_0, sigma_0)</code> is.  
Correspondingly, <code>normal_probability_above(529.5, mu_0, sigma_0)</code> is a better estimate of the probability of seeing at least 530 heads.

One way to convince yourself that this is a sensible estimate is with a simulation:

In [70]:
extreme_value_count = 0
for _ in range(100000):
    # count number of heads in 1000 flips
    num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        # and count how often the number is 'extreme'
        extreme_value_count += 1
        
print extreme_value_count / 100000

0.06155


Since the p-value is greater than our 5% significance, we <em>would not reject</em> the null hypothesis.  
If instead we saw 532 heads, the p-value would be:

In [71]:
two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

which is smaller than the 5% significance, which means we <em> would reject</em> the null hypothesis.  
It's the exact same test as before, just a different way of approaching the statistics.

Similarly, we would have:

In [72]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

For our one-sided test, if we saw 525 heads we would compute:

In [73]:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582083

which means we <em>would not reject</em> the null hypothesis.  
If we saw 527 heads, the computation would be:

In [74]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

and we <em>would reject</em> the null hypothesis.

#### Caution
Make sure your data is roughly normally distributed before using <code>normal_probability_above()</code> to compute p-values.  
The annals of bad data science are filled with examples of people opining that the chance of some observed event occurring at random is one in a million, when what they really mean is "the chance, assuming the data is distributed normally", which is pretty meaningless if the data is not distributed normally.  
There are various statistical tests for normality, but even plotting the data is a good start.

## Confidence Intervals

We have been testing hypotheses about the value of the heads probability $p$, which is a $parameter$ of the unknown 'heads' distribution.   
When this is the case, a third approach is to construct a <em>confidence interval</em> around the observed value of the parameter.  
For example, we can estimate the probability of the unfair coin by looking at the average value of the Bernoulli variables corresponding to each flip -- 1 if heads, 0 if tails.  
If we observe 525 heads out of 1000 flips, then we estimate $p$ equals 0.525.  
How confident can we be about this estimate?  
Well, if we knew the exact value of $p$, the Central Limit Theorem tells us that the average of those Bernoulli variables should be approximately normal, with mean $p$ and standard deviation <code>math.sqrt(p * (1 - p) / 1000)</code>.  
Here, we don't know $p$, so instead we use our estimate:

In [75]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
sigma

0.015791611697353755

This is not entirely justified, but people seem to do it anyway. (???)  
Using the normal approximation, we conclude that we are "95 percent confident" that the following interval contains the parameter $p$:

In [76]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

Note:  
This is a statement about the $interval$, not about $p$.  
You should understand it as the assertion that if you were to repeat the experiment many times, 95% of the time the "true" parameter (which is the same every time) would lie within the observed confidence interval (which might be different every time).

In particular, we do not conclude that the coin is unfair, since 0.5 falls within our confidence interval.  
If instead we had seen 540 heads, then we would have:

In [77]:
p_hat = 540 / 1000
mu = p_hat 
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print "Sigma is {}".format(sigma)
normal_two_sided_bounds(0.95, mu, sigma)

Sigma is 0.0157607106439


(0.5091095927295919, 0.5708904072704082)

Here, "fair coin", doesn't lie in the confidence interval.  
In other words, the "fair coin" hypothesis doesn't pass a test that you would expect it to pass 95% of the time if it were true.

## P-hacking

A procedure that erroneously rejects the null hypothesis only 5% of the time will -- by definition -- 5% of the time erroneouly reject the null hypothesis:

In [78]:
def run_experiment():
    """ Flip a fair coin 1000 times,  True=heads, False=tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """ Using the 5% significance levels """
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])
print num_rejections

46


Get all that?  
What it means is that if you are setting out to find "significant" results, you usually can.  
Test enough hypotheses against your data set, and one of them will almost certainly appear significant.  
Remove the right outliers, and you can probably get your $p$ value below 0.05.  
This is called 'P-hacking' and is in some ways a consequence of the "inference from p-values framework" that is discussed [here](http://ist-socrates.berkeley.edu/~maccoun/PP279_Cohen1.pdf).  
If you want to do good science, you should:
- determine your hypotheses before looking at the data
- clean your data without the hypothesis in mind
- keep in mind that p-values are not substitutes for common sense  

(an alternative is Bayesian Inference, which is discussed a bit later in the chapter).

## Example: Running an A/B Test

Something from the real world -- trying to get people to click on ads.  
Being a scientist, you decide to run an experiment by randomly showing site visitors one of two advertisements and tracking how many people click on each one.  
If 990 out of 1000 viewers of ad A click their ad while only 10 out of 1000 viewers of ad B click theirs, you can be pretty confident that A is the better ad.  
But what if the differences are not so stark?  
Here is where you would use statistical inference.

Let's say that $N_A$ people see ad A and that $n_A$ of them click it.  
We can think of each ad view as a Bernoulli trial where $P_A$ is the probability that someone clicks ad A.  
Then (if $N_A$ is large, which it is here) we know that $n_A / N_A$ is approximately a a normal random variable with mean $p_A$ and standard deviation $\large\sigma_A = {\sqrt {p_A(1 - p_A) / N_A}}$.

Similarly, $n_B/N_B$ is approxiamtely a normal random variable with mean $p_B$ and standard deviation  
$\large\sigma_B = {\sqrt{p_B(1 - p_B)/N_B}}$

In [79]:
def estimated_parameters(N, n):
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

If we assume those two normals are independent (which seems reasonable, since the individual Bernoulli trials ought to be), then their difference should also be normal with mean $p_B - p_A$ and standard deviation  
$\large{\sqrt{\sigma^2_A + \sigma^2_B}}$  

**Note**  
This is sort of cheating.  
The math only works out exacly like this if you *know* the standard deviations.  
Here, we are estimating them from the data, which means that we really should be using a *t*-distribution.  
However, for large enough data sets, it's close enough that it doesn't make much of a difference.  

This means that we can test the *null hypothesis* that $P_A$ and $P_B$ are the same thing (that is, $P_A$ and $P_B$ is zero), using the statistic:

In [80]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

which should approximately be a standard normal.

For example, if ad A gets 200 clicks out of 1000 views and ad B gets 180 clicks out of 1000 views, the statistic equals:

In [81]:
z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

The probability of seeing such a large difference if the means were actually equal would be:

In [82]:
two_sided_p_value(z)

0.254141976542236

which is large enough that you can't conclude that there is much of a difference.  
On the other hand, if ad B only got 150 clicks (compared to ad A's 200), we would have:

In [83]:
z = a_b_test_statistic(1000, 200, 1000, 150)
z

-2.948839123097944

In [84]:
two_sided_p_value(z)

0.003189699706216853

which means that there is only a 0.003 probability that you would see such a large difference if the ads were equally effective.

## Bayesian Inference

The procedures we have looked at so far have invloved making probability statements about our tests, for example:  
"There is only a 3% chance that you would observe such an extreme statistic if our null hypothesis were true."

An alternative approach to inference involves treating the unknown parameters themselves as random variables.  
The analyst (that's you) starts with a *prior distribution* for the parameters and then uses the observed data and Bayes' Theorem to get an updated *posterior distibution* for the parameters.  
Rather than making probability judgements about the tests, you make probability judgements abiout the parameters themselves.

For example, when the unknown parameter is a probability (as in our coin-flipping example), we often use a prior from the *Beta distribution*, which puts all of its probability between 0 and 1:

In [85]:
def B(alpha, beta):
    """ a normalizing constant so that the total probability is 1 """
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    # no weight outside of [0, 1]
    if x < 0 or x > 1:
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Generally speaking, this distribution centers its weight at:  
<code>alpha / (alpha + beta)</code>  
and the larger <code>alpha</code> and <code>beta</code> are, the "tighter", the distribution is.

For example, if <code>alpha</code> and <code>beta</code> are both 1, it's just the uniform distribution (centered at 0.5, very dispersed).  
If <code>alpha</code> is much larger than <code>beta</code>, most of the weight is near 1.  
If <code>alpha</code> is much smaller than <code>beta</code>, most of the weight is near zero.

What's interesting is that this allows us to make probability statements about hypotheses:  
"Based on the prior and the observed data, there is only a 5% likelihood that the coin's heads probability is between 49% and 51%".  
This is philosophically very different from a statement like:  
"If the coin were fair we would expect to observe data so extreme only 5% of the time".

**Note**  
Using Bayesian inference is considered somewhat controversial -- in part because its mathematics can get complicated and in part because of the subjective nature of choosing a prior.